### Elastichsearch

1. Запускаем 
```bash
docker run -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.3.2
```
2. Ставим клиент для Python 
```bash
pip3 install -U elasticsearch-dsl
```
3. Соединяемся

Альтернативно можно: 
1. Скачать:
https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.3.2-{linux|windows}-x86_64.tar.gz
2. Распаковать
3. Запустить `bin\elasticseach`

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Index, Document, analyzer, Integer, Text, Keyword

In [2]:
es = Elasticsearch(hosts=['localhost'])
test_index = Index('test_index', using=es)

test_index.delete(ignore=404)
test_index.create()

In [3]:
simple = analyzer('simple',
    tokenizer="standard",
    filter=["russian_stop", "lowercase", "russian_stemmer"],
)

In [4]:
@test_index.document
class MyDocument(Document):
    text_id = Integer()
    title = Text(analyzer=simple)
    text = Text(analyzer=simple)

In [5]:
import tqdm
import zipfile
import codecs

def read_texts(fn='data/texts.zip'):
    with zipfile.ZipFile(fn) as zf:
        with zf.open('texts.txt') as f:
            for idx, line in tqdm.tqdm(enumerate(codecs.iterdecode(f, 'utf-8'))):                                
                my_doc = MyDocument(text_id=idx, title='Document #%d'% idx, text=line) 
                my_doc.save()
                
                
read_texts()            

3190it [00:52, 60.32it/s]


In [6]:
for m in MyDocument.search().query('match', text='привет').execute():
    print(m.text[:20])

1231	 обрывки Выхожу
720	 Не стало Юрия С
1287	 Провидец Преди
808	 Умница Mr. Free
14	 12 миллионов бес
135	 Ах , какой был 
330	 Человек и кошка
2007	 Абсурдотека:Вр
449	 31 января 2006 


In [16]:
res = es.search(index='test_index', doc_type='doc', q='привет')
print('Hits: {}'.format(res['hits']['total']['value']))

Hits: 9
